In [1]:
!nvidia-smi

Sun May  7 06:35:24 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    41W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd ./drive/MyDrive/칠칠맞조/DL_project

/content/drive/MyDrive/칠칠맞조/DL_project


In [4]:
!git clone https://github.com/elyha7/yoloface

fatal: destination path 'yoloface' already exists and is not an empty directory.


In [5]:
%cd ./yoloface

/content/drive/MyDrive/칠칠맞조/DL_project/yoloface


In [6]:
# !pip install paddlepaddle-gpu==2.1.2
# !pip install insightface==0.2.1
# !pip install onnxruntime
!pip install facenet-pytorch
# !pip install -r requirements.txt
!pip install thop
!pip install face_detector

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 105.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 106.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 101.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 103.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 17.1 MB/s 

In [7]:
import torch 
import torch.nn as nn
import numpy as np

In [37]:
import os
import shutil
import glob
import cv2
from google.colab.patches import cv2_imshow

In [9]:
import numpy as np
from PIL import Image
import cv2

In [10]:
import pickle

In [11]:
from torchvision import transforms, utils, datasets, models

In [12]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


**Load model**

In [13]:
from facenet_pytorch import InceptionResnetV1
from face_detector import YoloDetector

In [ ]:
help(YoloDetector)

In [ ]:
help(InceptionResnetV1)

**Define methods**

In [14]:
resnet = InceptionResnetV1(pretrained='vggface2', device=device).eval()

  0%|          | 0.00/107M [00:00<?, ?B/s]

In [15]:
model = YoloDetector(target_size = 720, device = "cuda:0",min_face = 20)

cuda:0
/content/drive/MyDrive/칠칠맞조/DL_project/yoloface/weights/yolov5n_state_dict.pt


In [16]:
path = '/content/drive/MyDrive/DLpj/model_for_inference/model_v2.pt'

In [17]:
#defining the network

class Tuning(nn.Module):

  def __init__(self):
    super(Tuning,self).__init__()
    self.classifier = nn.Sequential(
            nn.Linear(512, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(128, 2),
            nn.Softmax(dim=1)

    )

    
    # self.classifier = nn.Sequential(
    #         nn.Linear(512, 2),
    #         nn.Softmax(dim=1)
          
    # )


  def forward(self,x):
    x = resnet(x)
    x = self.classifier(x)
    return x


model_dl = Tuning().to(device) ## GPU 사용

In [18]:
model_dl.load_state_dict(torch.load(path))
model_dl.eval()

Tuning(
  (classifier): Sequential(
    (0): Linear(in_features=512, out_features=128, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=128, out_features=2, bias=True)
    (4): Softmax(dim=1)
  )
)

utility methods

In [19]:
def detection(img, show=False, save_path=None, img_num=None):
  bboxes, points = model.predict(img)
  # crop and align image
  faces = model.align(img, points[0])
  # show pictures
  if show == True:
    for face in faces:
      cv2_imshow(face)  
  # save pictures
  if save_path != None:
    for i in range(len(faces)):
      img = faces[i]
      img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      img = Image.fromarray(img)
      img.save(save_path + f'/crop{img_num}_{i}.png')     
  # Reshape tensor for resnet module
  faces = torch.tensor(faces)
  if faces.dim() == 4:
    faces = faces.permute(0, 3, 1, 2)
    faces = faces.float()
  #bboxes = [float(num) for num in bboxes]
    return faces, bboxes, points
  else:
    return None, None, None

In [ ]:
def get_embeddings(faces):
    faces = faces.to(device)
    unknown_embeddings = resnet(faces).detach().cpu()
    return unknown_embeddings

In [ ]:
def recognition(face_db, unknown_embeddings, recog_thr) : 
    face_ids = []
    probs = []
    for i in range(len(unknown_embeddings)):
        result_dict = {}
        eb = unknown_embeddings[i]
        for name in face_db.keys():
            knownfeature_list = face_db[name]
            prob_list = []
            for knownfeature in knownfeature_list:
                prob = (eb - knownfeature).norm().item()
                prob_list.append(prob)
                if prob < recog_thr:
                    # 기준 넘으면 바로 break해서 같은 인물 계속 안 체크하도록
                    break
            result_dict[name] = min(prob_list)
        results = sorted(result_dict.items(), key=lambda d:d[1])
        result_name, result_probs = results[0][0], results[0][1]
        if result_probs < recog_thr: 
            face_ids.append(result_name)
        else : 
            face_ids.append('unknown')
        probs.append(result_probs)
    return face_ids, probs

In [ ]:
def recognition_v2(face_db, unknown_embeddings, recog_thr) : 
    face_ids = []
    similarities = []
    for i in range(len(unknown_embeddings)):
        result_dict = {}
        eb = unknown_embeddings[i]
        for name in face_db.keys():
            knownfeature_list = face_db[name]
            similarity_list = []
            for knownfeature in knownfeature_list:
                similarity =  torch.nn.functional.cosine_similarity(eb, knownfeature, dim=0)
                similarity_list.append(similarity)
                if similarity > recog_thr:
                    # 기준 넘으면 바로 break해서 같은 인물 계속 안 체크하도록
                    break
            result_dict[name] = max(similarity_list)
        results = sorted(result_dict.items(), key=lambda d:d[1], reverse=True)
        result_name, result_similarity = results[0][0], results[0][1]
        if result_similarity > recog_thr: 
            face_ids.append(result_name)
        else : 
            face_ids.append('unknown')
        similarities.append(result_similarity)
    return face_ids, similarities

In [ ]:
def recognition_v3(face_db, unknown_embeddings, recog_thr) : 
    face_ids = []
    probs = []
    for i in range(len(unknown_embeddings)):
        result_dict = {}
        eb = unknown_embeddings[i]
        for name in face_db.keys():
            knownfeature_list = face_db[name]
            prob_list = []
            for knownfeature in knownfeature_list:
                prob = (eb - knownfeature).norm().item()
                prob_list.append(prob)
            result_dict[name] = np.mean(prob_list)
        results = sorted(result_dict.items(), key=lambda d:d[1])
        result_name, result_probs = results[0][0], results[0][1]
        if result_probs < recog_thr: 
            face_ids.append(result_name)
        else : 
            face_ids.append('unknown')
        probs.append(result_probs)
    return face_ids, probs

In [ ]:
def preprocess(img, target, recog_thr, version) :
  faces, bboxes, _ = detection(img)

  unknown_embeddings = get_embeddings(faces)
  if version == 1:
    face_ids, probs = recognition(target, unknown_embeddings, recog_thr)
  if version == 2:
    face_ids, probs = recognition_v2(target, unknown_embeddings, recog_thr)
  if version == 3:
    face_ids, probs = recognition_v3(target, unknown_embeddings, recog_thr)

  return face_ids, probs


In [23]:
def DrawRectImg(img, bboxes, face_ids):
    rect_color = (0, 0, 255) # BGR
    rect_color_ = (225, 0, 0)
    rect_thickness = 2 # 이미지 사이즈에 맞게 조절해야할지도
    font_scale = 1 # 위와 동일
    font_color = (0, 0, 255) # BGR
    font_color_ = (255, 0, 0)
    font_thickness = 1 # 위와 동일
    
    for (bbox, face_id) in zip(bboxes[0], face_ids):
        if face_id != 'unknown':
            # bbox: x0, y0, x1, y1
            bbox = np.round(bbox).astype(int)
            cv2.rectangle(img, (bbox[0], bbox[1]), (bbox[2], bbox[3]),
                            rect_color, rect_thickness)
            cv2.putText(img, face_id, (bbox[0], bbox[1]-5),
                            1, font_scale, font_color, font_thickness)
            
        if face_id == 'unknown':
            # bbox: x0, y0, x1, y1
            bbox = np.round(bbox).astype(int)
            cv2.rectangle(img, (bbox[0], bbox[1]), (bbox[2], bbox[3]),
                            rect_color_, rect_thickness)
            cv2.putText(img, face_id, (bbox[0], bbox[1]-5),
                            1, font_scale, font_color_, font_thickness)
    img_draw = img

    return img_draw

process method

In [ ]:
def process_image(img_list, target, recog_thr=0.6, version=1, view_sim=False): 
  results=[]
  probs = []

  for img in img_list:
    _, bboxes, points = detection(img)
    face_ids, prob = preprocess(img, target, recog_thr, version)
    result = DrawRectImg(img, bboxes, face_ids)
    results.append(result)
    probs.append(prob)

  if view_sim == True:
    return results, probs
  
  else:
    return results

In [81]:
def process_image_dl(img_list): 
  results=[]
  data_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize(224),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])


  for j, img in enumerate(img_list): 
    os.mkdir("./cropimg_tmp")
    _, bboxes, points = detection(img, show=False, save_path="./cropimg_tmp", img_num=j)
    
    crop_img_list = []
    target_path = "./cropimg_tmp/*.*"
    for file in sorted(glob.glob(target_path)):
      image_read = cv2.imread(file)
      crop_img_list.append(image_read)


    shutil.rmtree("./cropimg_tmp")

    transformed_crop = []
    for i in crop_img_list:
      i = data_transform(i)
      transformed_crop.append(i)
    
    crop_img_tensor = torch.stack(transformed_crop, dim=0)

    crop_img_tensor = crop_img_tensor.to(device)

    output = model_dl(crop_img_tensor)
    print(output)


    max_row_index = torch.argmax(output[:, 1])
    print(max_row_index)
    max_row_value = output[max_row_index, 1]
    print(max_row_value)

    face_ids = []

    for i in range(len(output)):
      if (i == max_row_index) & (max_row_value >= 0.4):
        face_ids.append("charm_zu")
      else:
        face_ids.append("unknown")

    # _, preds = torch.max(output, 1)

    # face_ids = []

    # for pred in preds:
    #   if pred == 0:
    #     face_ids.append("unknown")
    #   if pred == 1:
    #     face_ids.append("charm_zu")

  
    result = DrawRectImg(img, bboxes, face_ids)
    results.append(result)

  return results

**crop & embedding charm_zu**

crop charm_zu

In [ ]:
# select the path
target_path = "/content/drive/MyDrive/DLpj/train_target_2/*.*"
target_images = []
for file in glob.glob(target_path):
  image_read = cv2.imread(file)
  resize_img = cv2.resize(image_read, (244, 244))
  # conversion numpy array into rgb image to show
  target_images.append(resize_img)
  # # wait for 1 second
  # k = cv2.waitKey(1000)
  # # destroy the window
  # cv2.destroyAllWindows()
  

In [ ]:
len(target_images)

671

In [ ]:
target_list = []

for i in range(len(target_images)):
  img = target_images[i]
  x,_,_ = detection(img, False, "/content/drive/MyDrive/DLpj/aligned_charm_zu_", i)
  target_list.append(x)



crop celebrity

In [ ]:
# select the path
target_path = "/content/drive/MyDrive/DLpj/celebrity/*.*"
celebrity_images = []
for file in glob.glob(target_path):
  image_read = cv2.imread(file)
  # conversion numpy array into rgb image to show
  celebrity_images.append(image_read)
  # # wait for 1 second
  # k = cv2.waitKey(1000)
  # # destroy the window
  # cv2.destroyAllWindows()

In [ ]:
len(celebrity_images)

1123

In [ ]:
celebrity_list = []

for i in range(len(celebrity_images)):
  img = celebrity_images[i]
  x,_,_ = detection(img, False, "/content/drive/MyDrive/DLpj/aligned_celebrities_",i)
  celebrity_list.append(x)


embedding charm_zu




In [ ]:
charm_zu_list = []
for img in target_images:
  img = data_transform(img)
  charm_zu_list.append(img)


In [ ]:
charm_zu_list_1 = torch.stack(charm_zu_list[:128], dim=0)
charm_zu_list_2 = torch.stack(charm_zu_list[128: 256], dim=0)
charm_zu_list_3 = torch.stack(charm_zu_list[256: 512], dim=0)
charm_zu_list_4 = torch.stack(charm_zu_list[512: ], dim=0)



In [ ]:
charm_zu_list_1.shape

torch.Size([128, 3, 244, 244])

In [ ]:
charm_zu_vectors_1 = get_embeddings(charm_zu_list_1)
charm_zu_vectors_2 = get_embeddings(charm_zu_list_2)
charm_zu_vectors_3 = get_embeddings(charm_zu_list_3)
charm_zu_vectors_4 = get_embeddings(charm_zu_list_4)

In [ ]:
charm_zu_vectors = []
charm_zu_vectors.append(charm_zu_vectors_1)
charm_zu_vectors.append(charm_zu_vectors_2)
charm_zu_vectors.append(charm_zu_vectors_3)
charm_zu_vectors.append(charm_zu_vectors_4)
charm_zu_vectors = torch.cat(charm_zu_vectors, dim=0)

In [ ]:
charm_zu_vectors.shape

torch.Size([671, 512])

In [ ]:
face_db = {}
face_db['charm_zu'] = charm_zu_vectors

In [ ]:
face_db

{'charm_zu': tensor([[ 0.02271, -0.01327, -0.07570,  ..., -0.06731,  0.01824, -0.05364],
         [ 0.02554, -0.08008, -0.06858,  ...,  0.04977, -0.04158, -0.10501],
         [ 0.01715, -0.07630, -0.07068,  ...,  0.04113, -0.04455, -0.09908],
         ...,
         [ 0.01563, -0.01090, -0.07394,  ..., -0.06433,  0.04027, -0.06744],
         [ 0.03785, -0.03048, -0.09122,  ...,  0.01801, -0.02615, -0.08153],
         [ 0.02787,  0.01531, -0.03118,  ..., -0.04808,  0.05037, -0.03020]])}

Test

In [85]:
# select the path
test_path = "/content/drive/MyDrive/칠칠맞조/DL_project/test_image/*.*"
test_images = []
for file in glob.glob(test_path):
  image_read = cv2.imread(file)
  # conversion numpy array into rgb image to show
  test_images.append(image_read)
  # # wait for 1 second
  # k = cv2.waitKey(1000)
  # # destroy the window
  # cv2.destroyAllWindows()
  

In [ ]:
# result, probs = process_image(test_images, face_db, 1.11, 1, True)

In [ ]:
test_images

In [ ]:
result= process_image_dl(test_images)

In [87]:
for img in result:
  cv2_imshow(img)

Output hidden; open in https://colab.research.google.com to view.